In [1]:
from  datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder
#LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
import numpy as np, pandas as pd
import matplotlib.pyplot as plot 
import sklearn
import lightgbm as lgb
import warnings
warnings.filterwarnings("ignore")

import joblib

/Users/hshan/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [48]:
def load_df(prev_last, pred_period, is_train=True, store='CA_1'):
    
    cal_cat_cols = ['weekday', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    sales_cat_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
    price_cat_cols = ['store_id', 'item_id']
    
    calendars = pd.read_csv('/Users/hshan/Downloads/M5/calendar.csv')
    sales = pd.read_csv('/Users/hshan/Downloads/M5/sales_train_validation.csv')
    prices = pd.read_csv('/Users/hshan/Downloads/M5/sell_prices.csv')
    
    if not is_train:
        for i in range((prev_last+1), (prev_last+pred_period+1)):
            f_string = f'd_{i}'
            sales[f_string] = pd.Series()
    
    ind_var = ['id'] + sales_cat_cols
    val_var = [col for col in sales.columns if col.startswith('d_')]
    df = pd.melt(sales, id_vars = ind_var, value_vars = val_var, var_name = 'd', value_name='sales')
    df = df.merge(calendars, on = 'd', copy = False)
    df = df.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    df = df[df['store_id']==store]
    
    cat_cols = cal_cat_cols + sales_cat_cols
    label_encoder = LabelEncoder()
    for col in cat_cols:
        df[col] = df[col].fillna('').astype('category')
        df[col] = label_encoder.fit_transform(df[col])

    unused_cols = ['wm_yr_wk', 'weekday','store_id']
    
    df.drop(unused_cols, inplace = True, axis = 1)
    
    return (df)


In [49]:
def lag_features(df):
    '''max lag should not be exceeding 57 in this case'''
    num = [1,7,28]
    lags = num
    windows = num
    lag_cols = [f'lag_{lag}' for lag in lags]
    

    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[['id','sales']].groupby('id')['sales'].shift(lag)
        
    for window in windows:
        for lag, lag_col in zip(lags, lag_cols):
            mean_col = f'mean_{lag}_{window}'
            df[mean_col] = df[['id',lag_col]].groupby('id')[lag_col].transform(lambda x: x.rolling(window).mean())
    
    return (df)

In [4]:
def pca(x_features_df, n_components):
    pca = PCA(n_components)
    pca_cols = []
    for i in range(n_components):
        pca_col = f'pc_{i+1}'
        pca_cols.append(pca_col)
    p_components = pca.fit_transform(x_features_df)
    pca_features_df = pd.DataFrame(data = p_components, columns = pca_cols)
    
    return(pca_features_df)

In [50]:
pred_period = 56
train_start = 1
train_last = 1913
test_start = train_last + 1
test_last  = train_last + pred_period
pred_start = test_last + 1
pred_last = test_last + pred_period
seed = 1231

removed_cols = ['id', 'date', 'sales', 'd', 'wm_yr_wk', 'weekday']

In [153]:
df = load_df(train_last, pred_period, is_train=True, store='WI_3')

In [154]:
df = lag_features(df)

In [155]:
df.dropna(inplace=True)
y_target = df['sales']

In [156]:
df_cols = list(df.columns)
x_features =[]
for feature in df_cols:
    if feature not in removed_cols:
        x_features.append(feature)


In [72]:
param = {
    "objective" : "poisson",
    "metric" :"rmse",
    'boosting_type' : 'gbdt',
    "force_row_wise" : True,
    "learning_rate" : 0.075,
    #"sub_feature" : 0.8,
    "sub_row" : 0.75,
    "bagging_freq" : 1,
    "lambda_l2" : 1.5,
    "lambda_l1" : 0.5,
    "nthread" : 5,
    "metric": "rmse",
    'verbosity': -1,
    'num_iterations' : 2000,
    'num_leaves': 200,
    "min_data_in_leaf": 200,
}


In [73]:
categorical_cols = ['wday','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'] + \
    ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']

In [157]:
lgbm_train_data = lgb.Dataset(df[x_features], label = df['sales'])
# lgbm_valid_data = lgb.Dataset(tst[x_features], label = tst['sales'])

# del df

In [158]:
# model = lgb.train(param, train_set=lgbm_train_data, valid_sets=lgbm_valid_data, categorical_feature=categorical_cols, 
#             verbose_eval=100, early_stopping_rounds=120)
# model = lgb.train(param, train_set=lgbm_train_data, categorical_feature=categorical_cols)
model = lgb.train(param, train_set=lgbm_train_data)

In [159]:
filename = '/Users/hshan/Kaggle/model_WI_3.sav'
joblib.dump(model, filename)

['/Users/hshan/Kaggle/model_WI_3.sav']

In [18]:
# del lgbm_train_data, lgbm_valid_data, tr, tst

In [160]:
# model = joblib.load('/Users/hshan/Kaggle/model_no_val.sav')
df = load_df(train_last, pred_period, is_train=False, store='WI_3')

In [161]:
first_day = datetime(2016, 4, 25)
for delta in range(0, pred_period):
    predict_day = first_day + timedelta(days = delta)
    predict_df = df.loc[(pd.to_datetime(df.date) <= predict_day) & (pd.to_datetime(df.date) >= predict_day - timedelta(days=57))]
    
    predict_df = lag_features(predict_df)
    
    predict_df = predict_df.loc[pd.to_datetime(predict_df.date) == predict_day]
    predict_df= predict_df[x_features]
    result = model.predict(predict_df)
    
    df.loc[pd.to_datetime(df.date)==predict_day,'sales'] = result
    del predict_df
    print(predict_day)

2016-04-25 00:00:00
2016-04-26 00:00:00
2016-04-27 00:00:00
2016-04-28 00:00:00
2016-04-29 00:00:00
2016-04-30 00:00:00
2016-05-01 00:00:00
2016-05-02 00:00:00
2016-05-03 00:00:00
2016-05-04 00:00:00
2016-05-05 00:00:00
2016-05-06 00:00:00
2016-05-07 00:00:00
2016-05-08 00:00:00
2016-05-09 00:00:00
2016-05-10 00:00:00
2016-05-11 00:00:00
2016-05-12 00:00:00
2016-05-13 00:00:00
2016-05-14 00:00:00
2016-05-15 00:00:00
2016-05-16 00:00:00
2016-05-17 00:00:00
2016-05-18 00:00:00
2016-05-19 00:00:00
2016-05-20 00:00:00
2016-05-21 00:00:00
2016-05-22 00:00:00
2016-05-23 00:00:00
2016-05-24 00:00:00
2016-05-25 00:00:00
2016-05-26 00:00:00
2016-05-27 00:00:00
2016-05-28 00:00:00
2016-05-29 00:00:00
2016-05-30 00:00:00
2016-05-31 00:00:00
2016-06-01 00:00:00
2016-06-02 00:00:00
2016-06-03 00:00:00
2016-06-04 00:00:00
2016-06-05 00:00:00
2016-06-06 00:00:00
2016-06-07 00:00:00
2016-06-08 00:00:00
2016-06-09 00:00:00
2016-06-10 00:00:00
2016-06-11 00:00:00
2016-06-12 00:00:00
2016-06-13 00:00:00


In [65]:
def submission(result_df):
    '''result_df is the resulted dataframe from for looping in predicting, 
    it includes 56 samples before the first day of the prediction d_1914'''
    sub_df = df.loc[pd.to_datetime(df.date)>= first_day,['id','d','sales']]
    val_df = sub_df.loc[(pd.to_datetime(df.date)>= first_day)& (pd.to_datetime(df.date)< first_day+timedelta(days=28))]
    eval_df = sub_df.loc[pd.to_datetime(df.date)>= (first_day+timedelta(days=28))]
    
    col_v= list(val_df['d'].unique())
    col_e= list(eval_df['d'].unique())
    
    f_cols =[]
    for i in range(1, 29):
        f_col = f'F{i}'
        f_cols.append(f_col)
    
    val_df = val_df.set_index(["id", "d" ]).unstack()['sales'][col_v].reset_index()
    eval_df = eval_df.set_index(["id", "d" ]).unstack()['sales'][col_e].reset_index()
    
    val_df.columns=['id'] + f_cols
    eval_df.columns=['id'] + f_cols
    
    for i in range(0,len(eval_df)):
        eval_df['id'][i] = eval_df['id'][i].replace('validation','evaluation')
        final_sub = pd.concat([val_df, eval_df])
        
    return (final_sub)

In [67]:
ca_1=submission(df)

In [79]:
ca_2=submission(df)

In [89]:
ca_3=submission(df)

In [101]:
ca_4=submission(df)

In [111]:
tx_1=submission(df)

In [121]:
tx_2=submission(df)

In [131]:
tx_3=submission(df)

In [142]:
wi_1=submission(df)

In [152]:
wi_2=submission(df)

In [162]:
wi_3=submission(df)

In [169]:
store_sub = [ca_1,ca_2,ca_3,ca_4,tx_1,tx_2,tx_3,wi_1,wi_2,wi_3]
sub = pd.concat(store_sub)
sub.shape

(60980, 29)

In [170]:
sub.to_csv('/Users/hshan/Downloads/M5/model_sub6.csv', index=False)

In [ ]:
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

In [22]:
# model sub
param = {
    "objective" : "poisson",
    "metric" :"rmse",
    'boosting_type' : 'gbdt',
    "force_row_wise" : True,
    "learning_rate" : 0.075,
    #"sub_feature" : 0.8,
    "sub_row" : 0.75,
    "bagging_freq" : 1,
    "lambda_l2" : 1.5,
    "lambda_l1" : 0.5,
    "nthread" : 5,
    "metric": "rmse",
    'verbosity': -1,
    'num_iterations' : 2000,
    'num_leaves': 200,
    "min_data_in_leaf": 200,
}

In [171]:
submission = pd.read_csv('/Users/hshan/Downloads/M5/model_sub6.csv')

In [172]:
submission.loc[submission['id'].str.contains('CA_1'), 'F1'] *= 1.009
submission.loc[submission['id'].str.contains('CA_2'), 'F1'] *= 1.02
submission.loc[submission['id'].str.contains('CA_3'), 'F1'] *= 1.009
submission.loc[submission['id'].str.contains('CA_4'), 'F1'] *= 1.011

submission.loc[submission['id'].str.contains('TX_1'), 'F1'] *= 1.01
submission.loc[submission['id'].str.contains('TX_2'), 'F1'] *= 1.011
submission.loc[submission['id'].str.contains('TX_3'), 'F1'] *= 1.011

submission.loc[submission['id'].str.contains('WI_1'), 'F1'] *= 1.015
submission.loc[submission['id'].str.contains('WI_2'), 'F1'] *= 1.02
submission.loc[submission['id'].str.contains('WI_3'), 'F1'] *= 1.015


submission.loc[submission['id'].str.contains('CA_1'), 'F2'] *= 0.994
submission.loc[submission['id'].str.contains('CA_2'), 'F2'] *= 0.998
submission.loc[submission['id'].str.contains('CA_3'), 'F2'] *= 0.994
submission.loc[submission['id'].str.contains('CA_4'), 'F2'] *= 0.996

submission.loc[submission['id'].str.contains('TX_1'), 'F2'] *= 0.995
submission.loc[submission['id'].str.contains('TX_2'), 'F2'] *= 0.996
submission.loc[submission['id'].str.contains('TX_3'), 'F2'] *= 0.996

submission.loc[submission['id'].str.contains('WI_1'), 'F2'] *= 0.998
submission.loc[submission['id'].str.contains('WI_2'), 'F2'] *= 1
submission.loc[submission['id'].str.contains('WI_3'), 'F2'] *= 0.998


submission.loc[submission['id'].str.contains('CA_1'), 'F3'] *= 0.994
submission.loc[submission['id'].str.contains('CA_2'), 'F3'] *= 0.998
submission.loc[submission['id'].str.contains('CA_3'), 'F3'] *= 0.994
submission.loc[submission['id'].str.contains('CA_4'), 'F3'] *= 0.996

submission.loc[submission['id'].str.contains('TX_1'), 'F3'] *= 0.995
submission.loc[submission['id'].str.contains('TX_2'), 'F3'] *= 0.996
submission.loc[submission['id'].str.contains('TX_3'), 'F3'] *= 0.996

submission.loc[submission['id'].str.contains('WI_1'), 'F3'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F3'] *= 1.005
submission.loc[submission['id'].str.contains('WI_3'), 'F3'] *= 1


submission.loc[submission['id'].str.contains('CA_1'), 'F4'] *= 0.98
submission.loc[submission['id'].str.contains('CA_2'), 'F4'] *= 1.02
submission.loc[submission['id'].str.contains('CA_3'), 'F4'] *= 0.98
submission.loc[submission['id'].str.contains('CA_4'), 'F4'] *= 1

submission.loc[submission['id'].str.contains('TX_1'), 'F4'] *= 0.99
submission.loc[submission['id'].str.contains('TX_2'), 'F4'] *= 1
submission.loc[submission['id'].str.contains('TX_3'), 'F4'] *= 1

submission.loc[submission['id'].str.contains('WI_1'), 'F4'] *= 1.00
submission.loc[submission['id'].str.contains('WI_2'), 'F4'] *= 1.02
submission.loc[submission['id'].str.contains('WI_3'), 'F4'] *= 1.00


submission.loc[submission['id'].str.contains('CA_1'), 'F5'] *= 0.995
submission.loc[submission['id'].str.contains('CA_2'), 'F5'] *= 1
submission.loc[submission['id'].str.contains('CA_3'), 'F5'] *= 0.995
submission.loc[submission['id'].str.contains('CA_4'), 'F5'] *= 0.998

submission.loc[submission['id'].str.contains('TX_1'), 'F5'] *= 0.996
submission.loc[submission['id'].str.contains('TX_2'), 'F5'] *= 0.998
submission.loc[submission['id'].str.contains('TX_3'), 'F5'] *= 0.998

submission.loc[submission['id'].str.contains('WI_1'), 'F5'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F5'] *= 1.01
submission.loc[submission['id'].str.contains('WI_3'), 'F5'] *= 1


submission.loc[submission['id'].str.contains('CA_1'), 'F6'] *= 1.0002
submission.loc[submission['id'].str.contains('CA_2'), 'F6'] *= 1.001
submission.loc[submission['id'].str.contains('CA_3'), 'F6'] *= 1.0002
submission.loc[submission['id'].str.contains('CA_4'), 'F6'] *= 1.001

submission.loc[submission['id'].str.contains('TX_1'), 'F6'] *= 1.000
submission.loc[submission['id'].str.contains('TX_2'), 'F6'] *= 1.001
submission.loc[submission['id'].str.contains('TX_3'), 'F6'] *= 1.001

submission.loc[submission['id'].str.contains('WI_1'), 'F6'] *= 1.001
submission.loc[submission['id'].str.contains('WI_2'), 'F6'] *= 1.01
submission.loc[submission['id'].str.contains('WI_3'), 'F6'] *= 1.001


submission.loc[submission['id'].str.contains('CA_1'), 'F7'] *= 0.994
submission.loc[submission['id'].str.contains('CA_2'), 'F7'] *= 1
submission.loc[submission['id'].str.contains('CA_3'), 'F7'] *= 0.994
submission.loc[submission['id'].str.contains('CA_4'), 'F7'] *= 0.998

submission.loc[submission['id'].str.contains('TX_1'), 'F7'] *= 0.995635
submission.loc[submission['id'].str.contains('TX_2'), 'F7'] *= 0.998
submission.loc[submission['id'].str.contains('TX_3'), 'F7'] *= 0.998

submission.loc[submission['id'].str.contains('WI_1'), 'F7'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F7'] *= 1.01
submission.loc[submission['id'].str.contains('WI_3'), 'F7'] *= 1


submission.loc[submission['id'].str.contains('CA_1'), 'F8'] *= 0.996
submission.loc[submission['id'].str.contains('CA_2'), 'F8'] *= 1
submission.loc[submission['id'].str.contains('CA_3'), 'F8'] *= 0.996
submission.loc[submission['id'].str.contains('CA_4'), 'F8'] *= 0.998

submission.loc[submission['id'].str.contains('TX_1'), 'F8'] *= 0.9988
submission.loc[submission['id'].str.contains('TX_2'), 'F8'] *= 1
submission.loc[submission['id'].str.contains('TX_3'), 'F8'] *= 1

submission.loc[submission['id'].str.contains('WI_1'), 'F8'] *= 1
submission.loc[submission['id'].str.contains('WI_2'), 'F8'] *= 1.01
submission.loc[submission['id'].str.contains('WI_3'), 'F8'] *= 1


In [173]:
for i in range(9,20):
    if i!=11:
        #submission['F'+str(i)] *= 1.01 
        submission.loc[submission['id'].str.contains('CA_1'), 'F'+str(i)] *= 1
        submission.loc[submission['id'].str.contains('CA_2'), 'F'+str(i)] *= 1.015
        submission.loc[submission['id'].str.contains('CA_3'), 'F'+str(i)] *= 1
        submission.loc[submission['id'].str.contains('CA_4'), 'F'+str(i)] *= 1.013

        submission.loc[submission['id'].str.contains('TX_1'), 'F'+str(i)] *= 1
        submission.loc[submission['id'].str.contains('TX_2'), 'F'+str(i)] *= 1.013
        submission.loc[submission['id'].str.contains('TX_3'), 'F'+str(i)] *= 1.013

        submission.loc[submission['id'].str.contains('WI_1'), 'F'+str(i)] *= 1.015
        submission.loc[submission['id'].str.contains('WI_2'), 'F'+str(i)] *= 1.02
        submission.loc[submission['id'].str.contains('WI_3'), 'F'+str(i)] *= 1.015
        
for i in range(20,29):
    #submission['F'+str(i)] *= 1.02
    submission.loc[submission['id'].str.contains('CA_1'), 'F'+str(i)] *= 1.015
    submission.loc[submission['id'].str.contains('CA_2'), 'F'+str(i)] *= 1.03
    submission.loc[submission['id'].str.contains('CA_3'), 'F'+str(i)] *= 1.015
    submission.loc[submission['id'].str.contains('CA_4'), 'F'+str(i)] *= 1.025

    submission.loc[submission['id'].str.contains('TX_1'), 'F'+str(i)] *= 1.02
    submission.loc[submission['id'].str.contains('TX_2'), 'F'+str(i)] *= 1.025
    submission.loc[submission['id'].str.contains('TX_3'), 'F'+str(i)] *= 1.025

    submission.loc[submission['id'].str.contains('WI_1'), 'F'+str(i)] *= 1.03
    submission.loc[submission['id'].str.contains('WI_2'), 'F'+str(i)] *= 1.04
    submission.loc[submission['id'].str.contains('WI_3'), 'F'+str(i)] *= 1.03
    print(submission['F'+str(i)].sum())
    
    


114355.66201856881
119442.08301150176
90030.69711273958
83256.40634146568
83523.20867694943
79813.90486883943
91404.69892056477
106857.919951501
102881.49510454017


In [174]:
submission.to_csv('/Users/hshan/Downloads/M5/model_sub66.csv')